# NB_3_Silver

In [1]:
# Import Packages
import pandas as pd
from datetime import datetime

In [2]:
# Logging Variables
current_time = datetime.now()
run_id = current_time.strftime("%Y%m%d_%H%M%S")

In [3]:
# Run common files
%run Notebooks/NB_1_Settings.ipynb
%run Notebooks/NB_1_Common_Functions.ipynb

In [4]:
# Import data from bronze
df_races = func_read_csv('races.csv', v_bronze_path)
df_results = func_read_csv('results.csv', v_bronze_path)

Reading CSV from C:\Users\Joe.Pilny\engineering-recruitment-assignments\data-engineering\datapipeline\results\bronze\races.csv
Reading CSV from C:\Users\Joe.Pilny\engineering-recruitment-assignments\data-engineering\datapipeline\results\bronze\results.csv


In [5]:
# Handle nulls in time column
df_races['time'] = df_races['time'].fillna('00:00:00')

# Convert and merge columns
df_races['datetime'] = pd.to_datetime(df_races['date'] + ' ' + df_races['time'], format = '%Y-%m-%d %H:%M:%S')
df_races['datetime'] = df_races['datetime'].dt.strftime('%Y-%m-%dT%H:%M:%S.%f').str[:-3]

In [6]:
# Change type on key columns
df_results['position'] = df_results['position'].astype('Int64')

In [7]:
# Select Required fields
df_races_select = df_races[['raceId', 'year', 'round', 'name', 'datetime']]
df_results_select = df_results[['raceId', 'driverId', 'position', 'fastestLapTime']]

# Set indexes
df_races_indexed = df_races_select.set_index('raceId')
df_results_indexed = df_results_select.set_index('raceId')

# Join Data Sets
df_stats = df_results_indexed.join(df_races_indexed, how = 'left')

In [8]:
df_race_winners = df_stats[df_stats['position'] == 1]
df_race_winners_final = df_race_winners.drop('position', axis = 1)

In [9]:
# Write to silver
func_write_csv(df_stats, v_silver_path, 'stats.csv')
func_write_csv(df_race_winners, v_silver_path, 'race_winners.csv')

Writing CSV to C:\Users\Joe.Pilny\engineering-recruitment-assignments\data-engineering\datapipeline\results\silver\stats.csv
Writing CSV to C:\Users\Joe.Pilny\engineering-recruitment-assignments\data-engineering\datapipeline\results\silver\race_winners.csv
